In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel_transfer, find_state, directory_scratch, directory_project, directory_archive
import dropbox
from amftrack.transfer.functions.transfer import upload
from subprocess import call

In [13]:
directory = directory_project
directory = '/run/user/357100554/gvfs/smb-share:server=sun.amolf.nl,share=shimizu-data,user=bisot/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/'

update_plate_info(directory)

analysed: 100%|██████████████████████████████████████████████████████████████████| 10100/10100 [01:20<00:00, 124.96it/s]


FileMetadata(client_modified=datetime.datetime(2021, 12, 24, 9, 24, 42), content_hash='33594f0cafeb71775db497ad8565c4ac886e6f6d393fb40e70ba503361f6f16e', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:mcSwhUrTbbAAAAAAAAAFYg', is_downloadable=True, media_info=NOT_SET, name='data_info.json', parent_shared_folder_id=NOT_SET, path_display='/data_info.json', path_lower='/data_info.json', property_groups=NOT_SET, rev='5d3e0ecd2e5208b8306a1', server_modified=datetime.datetime(2021, 12, 24, 9, 24, 42), sharing_info=NOT_SET, size=5758908, symlink_info=NOT_SET)


In [14]:
infos = get_data_info()

In [18]:
infos['total_path'].iloc[0]

'/run/user/357100554/gvfs/smb-share:server=sun.amolf.nl,share=shimizu-data,user=bisot/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/20211014_1503_Plate17'

In [37]:
API = str(np.load(os.getenv('HOME')+'/pycode/API_drop.npy'))

def get_current_folders(directory):
    plate_info = get_data_info()
    if directory == 'dropbox':
        dbx = dropbox.Dropbox(API)
        response = dbx.files_list_folder("",recursive = True)
        listdir = [file.name.split(".")[0] for file in response.entries]
        return(plate_info.loc[np.isin(plate_info['folder'],listdir)])
    else:
        listdir = os.listdir(directory)
        return(plate_info.loc[np.isin(plate_info['folder'],listdir)&(plate_info['total_path']==directory+plate_info['folder'])])

In [38]:
get_current_folders('dropbox')

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp
17,/run/user/357100554/gvfs/smb-share:server=sun....,94,39,Carrot,A5,001P100N,Y,-,20201123,True,...,True,True,False,True,True,True,True,"26.11.2020, 03:02:",20201126_0302_Plate39,25
47,/run/user/357100554/gvfs/smb-share:server=sun....,94,39,Carrot,A5,001P100N,Y,-,20201123,True,...,True,True,False,True,True,True,False,"28.11.2020, 01:12:",20201128_0112_Plate39,25
138,/run/user/357100554/gvfs/smb-share:server=sun....,94,39,Carrot,A5,001P100N,Y,-,20201123,True,...,True,True,False,True,True,True,True,"25.11.2020, 01:02:",20201125_0102_Plate39,25
204,/run/user/357100554/gvfs/smb-share:server=sun....,94,39,Carrot,A5,001P100N,Y,-,20201123,True,...,True,True,False,True,True,True,True,"26.11.2020, 02:02:",20201126_0202_Plate39,25
249,/run/user/357100554/gvfs/smb-share:server=sun....,94,39,Carrot,A5,001P100N,Y,-,20201123,True,...,True,True,False,True,True,True,False,"28.11.2020, 00:12:",20201128_0012_Plate39,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5704,/run/user/357100554/gvfs/smb-share:server=sun....,94,39,Carrot,A5,001P100N,Y,-,20201123,True,...,True,True,False,True,True,True,True,"27.11.2020, 16:13:",20201127_1613_Plate39,25
5718,/run/user/357100554/gvfs/smb-share:server=sun....,94,39,Carrot,A5,001P100N,Y,-,20201123,True,...,True,True,False,True,True,True,True,"26.11.2020, 20:09:",20201126_2009_Plate39,25
5732,/run/user/357100554/gvfs/smb-share:server=sun....,94,39,Carrot,A5,001P100N,Y,-,20201123,True,...,True,True,False,True,True,True,True,"25.11.2020, 13:02:",20201125_1302_Plate39,25
5820,/run/user/357100554/gvfs/smb-share:server=sun....,94,39,Carrot,A5,001P100N,Y,-,20201123,True,...,True,True,False,True,True,True,True,"24.11.2020, 12:02:",20201124_1202_Plate39,25


In [27]:
all_folders = get_current_folders(directory)
folders = all_folders.loc[all_folders['Plate']==94]

In [4]:
dir_drop = 'prince_data'

In [5]:
directory_targ = os.path.join(directory_scratch,'temp')+'/'
# directory_targ = os.path.join(directory_archive,'Prince_Data')+'/'


In [11]:
%autoreload 2
from amftrack.transfer.functions.transfer import download, zip_file,unzip_file

API = str(np.load('/home/cbisot/pycode/API_drop.npy'))
source = f'data_info.json'
target = '/scratch-shared/amftrack/temp/data_info.json'
# download(API,source,target,end='')
upload(API,target,f"/{source}", chunk_size=256 * 1024 * 1024,
)

FileMetadata(client_modified=datetime.datetime(2021, 12, 23, 8, 55, 46), content_hash='c47ecb86baa70489f9978b728e675c048cca2625e5bb4da6e128586c2abd6e0d', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:mcSwhUrTbbAAAAAAAAAFcg', is_downloadable=True, media_info=NOT_SET, name='data_info.json', parent_shared_folder_id=NOT_SET, path_display='/data_info.json', path_lower='/data_info.json', property_groups=NOT_SET, rev='5d3cc677af9368b8306a1', server_modified=datetime.datetime(2021, 12, 23, 8, 55, 46), sharing_info=NOT_SET, size=1487862, symlink_info=NOT_SET)


In [12]:
dbx = dropbox.Dropbox(API)

In [ ]:
dropbox.files.CommitInfo(path=target_path)

**To transfer data from surfsara to dropbox**
- use the 'toward_drop.py' function including the folders of interest and using dir_drop as a parameter indicating where in dropbox the data should go


In [6]:
dir_drop = 'prince_data'
run_parallel_transfer('toward_drop.py',[directory,dir_drop],folders,10,'10:00','staging',cpus = 1,node = 'staging')

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 242716
Submitted batch job 242717
Submitted batch job 242718
Submitted batch job 242719


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 242720
Submitted batch job 242721
Submitted batch job 242722
Submitted batch job 242723
Submitted batch job 242724


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


...

Submitted batch job 242739
Submitted batch job 242740
Submitted batch job 242741
Submitted batch job 242742
Submitted batch job 242743


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 242744
Submitted batch job 242745
Submitted batch job 242746
Submitted batch job 242747
Submitted batch job 242748


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 242749


**To transfer data from dropbox to surfsara**
- use the 'from_drop.py' function including the folders of interest


In [7]:
unzip = 'False'
flatten = False
run_parallel_transfer('from_drop.py',[directory_targ,dir_drop,unzip,flatten],folders,5,'10:00','staging',cpus = 1,node = 'staging')

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 242758
Submitted batch job 242759
Submitted batch job 242760
Submitted batch job 242761
Submitted batch job 242762


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 242763
Submitted batch job 242764
Submitted batch job 242765
Submitted batch job 242766
Submitted batch job 242767


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 242781
Submitted batch job 242782
Submitted batch job 242783
Submitted batch job 242784


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 242785
Submitted batch job 242786
Submitted batch job 242787
Submitted batch job 242788
Submitted batch job 242789
Submitted batch job 242790
Submitted batch job 242791


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [12]:
must_zip = True
must_unzip = False
run_parallel_transfer('within_surf.py',[directory,directory_targ,must_zip,must_unzip],folders,5,'10:00','staging',cpus = 1,node = 'staging')

Submitted batch job 242022


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [7]:
plates = set(folders['Plate'].values)

In [10]:
call(f"tar cvf /archive/cbisot/prince_data/94_20201123.tar {directory_targ}94_20201123*", shell=True)

tar: /archive/cbisot/prince_data/94_20201123.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


2

In [ ]:
def transfer_parallel_whole_plate(code, args, folders, num_parallel, time, name,cpus = 128,node = 'thin'):
    op_id = time_ns()
    folders.to_json(f'{directory_scratch}temp/{op_id}.json')# temporary file
    plates = set(folders['Plate'].values)
    length = len(plates)
    begin_skel = 0
    end_skel = length // num_parallel + 1
    args_str = [str(arg) for arg in args]
    arg_str = " ".join(args_str)
    arg_str_out = "_".join([str(arg) for arg in args if type(arg)!=str])
    for j in range(begin_skel, end_skel):
        start = num_parallel * j
        stop = num_parallel * j + num_parallel - 1
        ide = time_ns()
        my_file = open(path_job, "w")
        my_file.write(
            f"#!/bin/bash \n#Set job requirements \n#SBATCH --nodes=1 \n#SBATCH -t {time}\n #SBATCH --ntask=1 \n#SBATCH --cpus-per-task={cpus}\n#SBATCH -p {node} \n"
        )
        my_file.write(
            f'#SBATCH -o "{path_code}slurm/{name}_{arg_str_out}_{start}_{stop}_{ide}.out" \n'
        )
        my_file.write(f"source /home/cbisot/miniconda3/etc/profile.d/conda.sh\n")
        my_file.write(f"conda activate amftrack\n")
        my_file.write(f"for i in `seq {start} {stop}`; do\n")
        my_file.write(f"\t python {path_code}amftrack/pipeline/scripts/image_processing/{code} {arg_str} {op_id} $i &\n")
        my_file.write("done\n")
        my_file.write("wait\n")
        my_file.close()
        call(f"sbatch {path_job}", shell=True)

In [8]:
def run_parallel_transfer(code, args, folders, num_parallel, time, name,cpus = 1,node = 'staging',name_job='transfer.sh'):
    path_job = f'{path_bash}{name_job}'
    op_id = time_ns()
    folders.to_json(f'{directory_scratch}temp/{op_id}.json')# temporary file
    length = len(folders)
    begin_skel = 0
    end_skel = length // num_parallel + 1
    args_str = [str(arg) for arg in args]
    arg_str = " ".join(args_str)
    arg_str_out = "_".join([str(arg) for arg in args if type(arg)!=str])
    for j in range(begin_skel, end_skel):
        start = num_parallel * j
        stop = num_parallel * j + num_parallel - 1
        ide = time_ns()
        my_file = open(path_job, "w")
        my_file.write(
            f"#!/bin/bash \n#Set job requirements \n#SBATCH --nodes=1 \n#SBATCH -t {time}\n #SBATCH --ntask=1 \n#SBATCH --cpus-per-task={cpus}\n#SBATCH -p {node} \n"
        )
        my_file.write(
            f'#SBATCH -o "{path_code}slurm/{name}_{arg_str_out}_{start}_{stop}_{ide}.out" \n'
        )
        my_file.write(f"source /home/cbisot/miniconda3/etc/profile.d/conda.sh\n")
        my_file.write(f"conda activate amftrack\n")
        my_file.write(f"for i in `seq {start} {stop}`; do\n")
        my_file.write(f"\t python {path_code}amftrack/transfer/scripts/{code} {arg_str} {op_id} $i &\n")
        my_file.write("done\n")
        my_file.write("wait\n")
        my_file.close()
        call(f"sbatch {path_job}", shell=True)